In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
import numpy as np

In [37]:
df = pd.read_csv('data/train.csv', index_col=['id'], parse_dates=['fecha'], error_bad_lines=False)

df = df.dropna()
#df = df.sample(2000)

In [38]:
df.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [39]:
#create a dataframe with all training data except the target column
X = df[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'gimnasio', 'usosmultiples', 'piscina']]
y = df['precio'].values

In [40]:
#split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [41]:
# Create KNN regressor
knn = KNeighborsRegressor(n_neighbors = 3)# Fit the classifier to the data
knn.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=3, p=2,
          weights='uniform')

In [42]:
#show first 5 model predictions on the test data
predicted = knn.predict(X_test)
predicted[0:5]

array([ 593333.33333333,  633333.33333333, 2213333.33333333,
        954000.        ,  981666.66666667])

In [43]:
#check MAE of our model on the test data
mean_absolute_error(predicted, y_test)

926749.9887741806

In [44]:
#create a new KNN model
knn_cv = KNeighborsRegressor(n_neighbors=3) #train model with cv of 5 
cv_scores = cross_val_score(knn_cv, X, y, cv=5) #print each cv score (accuracy) and average them
print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

[0.40943276 0.40328134 0.38525043 0.4006632  0.36875299]
cv_scores mean:0.3934761449045302


In [ ]:
#create new a knn model
knn2 = KNeighborsRegressor() #create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(3, 25)} #use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn2, param_grid, cv=5) #fit model to data
knn_gscv.fit(X, y)

In [ ]:
#check top performing n_neighbors value
knn_gscv.best_params_

In [35]:
#check mean score for the top performing value of n_neighbors
knn_gscv.best_score_

0.013